# Pipeline Pseudocode
## Function: Handle User Request
```python
def handle_user_request(user, query, target_waifu):
    # Step 1: Retrieve relevant data
    waifu_data = database[user][target_waifu]
    past_convo = waifu_data["past_conversations"]
    event_status = waifu_data["current_event_status"]
    last_contact_time = waifu_data["last_contact_time"]
    waifu_data["last_contact_time"] = time.now()

    # Step 2: Update event_status if the last contact exceeds the threshold
    if time.now() - last_contact_time > threshold_of_update:
        event_status = GPT_update(event_status, target_waifu, last_contact_time)

    # Step 3: Prepare the prompt for the LLM
    prompt = (
        f"You are {target_waifu}, you talked about {past_convo}, "
        f"your recent thoughts and events are {event_status}, "
        f"and here's the new input: {query}. Respond appropriately. "
        f"Additionally, after your response add a '$$' sign. "
        f"If you decide the event has changed, type the new events after the '$$' sign."
    )

    # Step 4: Generate response using LLM
    response = LLM(prompt)  # The LLM updates event_status implicitly if necessary

    # Step 5: Log the request and response
    waifu_data["past_conversations"].append({"query": query, "response": response})

    # Step 6: Save updated event_status to the database
    waifu_data["current_event_status"] = event_status
    database.save()

    return response

## Function: Handle Trigger Event
def handle_trigger_event(user, query, target_waifu, event_number):
    # Step 1: Retrieve relevant data
    waifu_data = database[user][target_waifu]
    past_convo = waifu_data["past_conversations"]
    event_status = waifu_data["current_event_status"]
    last_contact_time = waifu_data["last_contact_time"]

    # Step 2: Update event_status with the completed event
    event_status = GPT_update_completed_event(event_status, target_waifu, last_contact_time, event_number)

    # Step 3: Prepare the prompt for the LLM
    prompt = (
        f"Event {event_number} has been completed. Here is the updated context: {event_status}. "
        f"Additionally, {past_convo} was discussed earlier. New input: {query}. "
        f"Provide a response and, if applicable, update event details after the '$$' sign."
    )

    # Step 4: Generate response using LLM
    response = LLM(prompt)

    # Step 5: Log the trigger event
    waifu_data["event_status_log"].append({
        "user": user,
        "query": query,
        "target_waifu": target_waifu,
        "event_number": event_number
    })

    # Step 6: Save updated event_status to the database
    waifu_data["current_event_status"] = event_status
    database.save()

    return response


# Example usage

## For either case
```python
    response = handle_trigger_event(user, query, target_waifu, event_number)
    response = handle_user_request(user, query, target_waifu)

